In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/training_data/Dendritic_CapsNets/', '/content/training_data/')

In [ ]:
!pip install scikit-learn
!pip install joblib soundfile speechpy librosa matplotlib
!pip install joblib==1.2.0
!pip install np_utils

In [1]:
#from include.functions import initialise, load_data
#from include.stats import plot_accuracy_loss
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
#from keras.utils import np_utils
import joblib
import os
###################################################
def load_data(load_from_file,name):
    if load_from_file:
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = '/content/training_data'

        x_train = joblib.load(os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        x_test = joblib.load(os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        y_train = joblib.load(os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        y_test = joblib.load(os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        num_labels = joblib.load(os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #data = joblib.load(os.path.join(save_dir, 'data_'+name+'.joblib'))
        #labels = joblib.load(os.path.join(save_dir, 'labels_'+name+'.joblib'))
    else:
        #x_train, x_test, y_train, y_test, num_labels = extract_data()
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = current_dir + subdir
        joblib.dump(x_train, os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        joblib.dump(x_test, os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        joblib.dump(y_train, os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        joblib.dump(y_test, os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        joblib.dump(num_labels, os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #joblib.dump(data, os.path.join(save_dir, 'data_'+name+'.joblib'))
        #joblib.dump(labels, os.path.join(save_dir, 'labels_'+name+'.joblib'))
    return x_train,x_test,y_train,y_test,num_labels#,data, labels
###################################################
# Squash function
def squash(x, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(x), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + 1e-7)
    return scale * x

###################################################
# Define Capsule Layer
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(name='capsule_kernel',
                                      shape=(input_shape[-1], self.num_capsules * self.dim_capsule),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs):
        # Following is a simplified routing algorithm
        inputs = tf.keras.backend.reshape(inputs, (-1, inputs.shape[-1]))
        u = tf.keras.backend.dot(inputs, self.kernel)
        u = tf.keras.backend.reshape(u, (-1, self.num_capsules, self.dim_capsule))
        b = tf.zeros_like(u[:, :, 0])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            s = tf.reduce_sum(c[:, :, tf.newaxis] * u, axis=1)
            v = squash(s)
            b = b + tf.reduce_sum(u * v[:, tf.newaxis, :], axis=2)
        return v

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_capsules': self.num_capsules,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        })
        return config
###################################################
###################################################
# Custom Dendritic Layer
class DendriticLayer(layers.Layer):
    def __init__(self, units, segments):
        super(DendriticLayer, self).__init__()
        self.units = units
        self.segments = segments

    def build(self, input_shape):
        self.segment_weights = []
        self.segment_biases = []
        for _ in range(self.segments):
            self.segment_weights.append(self.add_weight(
                name='dend1',
                shape=(input_shape[-1], self.units),
                initializer='random_normal',
                trainable=True))
            self.segment_biases.append(self.add_weight(
                name='dend2',
                shape=(self.units,),
                initializer='zeros',
                trainable=True))

        self.soma_weight = self.add_weight(
            name='dend3',
            shape=(self.units * self.segments, self.units),
            initializer='random_normal',
            trainable=True)
        self.soma_bias = self.add_weight(
            name='dend4',
            shape=(self.units,),
            initializer='zeros',
            trainable=True)

    def call(self, inputs):
        outputs = []
        for i in range(self.segments):
            x = tf.matmul(inputs, self.segment_weights[i]) + self.segment_biases[i]
            x = tf.nn.relu(x)
            outputs.append(x)

        concat = tf.concat(outputs, axis=-1)
        soma_output = tf.matmul(concat, self.soma_weight) + self.soma_bias
        soma_output = tf.nn.relu(soma_output)
        return soma_output

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'segments': self.segments
        })
        return config

def folder_size():
     # import module
     import os

     # assign size
     size = 0

     # assign folder path
     Folderpath = "/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.tf"

     # get size
     for path, dirs, files in os.walk(Folderpath):
         for f in files:
             fp = os.path.join(path, f)
             size += os.path.getsize(fp)
             size /= 1000000
     # display size
     print("Folder size: " + str(size))



def Dendritic_CapsNets_example():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    except:
        print("*************Exception*************")
    print("*************START*************\n")
    #loadfromfile=False
    loadfromfile=True
    x_train,x_test,y_train,y_test,num_labels =load_data(load_from_file=loadfromfile,name= 'Dendritic_CapsNets')

    print("Before Reshape")
    #print("x_train.shape :" +  str(x_train.shape) +"    y_train.shape:"+ str(y_train.shape))
    #print("x_test.shape :" +  str(x_test.shape) +"    y_test.shape:"+ str(y_test.shape))

    print("Categorical:y_train:shape "+str(y_train.shape))
    print("Categorical:y_test:shape "+str(y_test.shape))
    print("Categorical:y_train:[0][1][2][3] "+str(y_train[0])+" "+str(y_train[1])+" "+str(y_train[2])+" "+str(y_train[3]))
    print("Categorical:y_test:[0][1][2][3] "+str(y_test[0])+" "+str(y_test[1])+" "+str(y_test[2])+" "+str(y_test[3]))

    from keras import utils
    y_train = utils.to_categorical(y_train)
    y_test = utils.to_categorical(y_test)
    in_shape = x_train[0].shape


    print("\n\nAfter Reshape")
    print("Categorical:y_train:shape "+str(y_train.shape))
    print("Categorical:y_test:shape "+str(y_test.shape))
    print("Categorical:y_train:[0][1][2][3] "+str(y_train[0])+" "+str(y_train[1])+" "+str(y_train[2])+" "+str(y_train[3]))
    print("Categorical:y_test:[0][1][2][3] "+str(y_test[0])+" "+str(y_test[1])+" "+str(y_test[2])+" "+str(y_test[3]))


    x_train = x_train.reshape(x_train.shape[0], in_shape[1], in_shape[0], 1)
    x_test = x_test.reshape(x_test.shape[0], in_shape[1], in_shape[0], 1)

    num_timesteps, num_features = x_train.shape[1], x_train.shape[2]
    num_classes = 8   #y1_train.shape[1]


    print("num_timesteps: "+str(num_timesteps))
    print("num_features: "+str(num_features))
    print("num_classes: "+str(num_classes))
    ###################################################
    # Define CNN with Dendritic Layer Followed by Capsule Layer
    import matplotlib.pyplot as plt

    input_shape = (num_timesteps, num_features, 1)
    inputs = tf.keras.Input(shape=input_shape)
    conv1 = layers.Conv2D(128, (3, 3), activation='relu')(inputs)
    conv2 = layers.Conv2D(32, (3, 3), activation='relu')(conv1)
    pooling1 = layers.MaxPooling2D(pool_size=2)(conv2)
    dropout1 = layers.Dropout(0.5)(pooling1)
    flatten1 = layers.Flatten()(dropout1)
    dendritic = DendriticLayer(64, 2)(flatten1)  # 64 units, 2 segments
    capsule = CapsuleLayer(num_capsules=10, dim_capsule=32, routings=3)(dendritic)
    #flatten2 = layers.Flatten()(capsule)
    dense1 = Dense(32, activation='relu')(capsule)
    dropout2 = layers.Dropout(0.5)(dense1)
    outputs = Dense(num_classes, activation='softmax')(dropout2)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile and Train
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
  #####################
    loss_history = []
    accuracy_history = []

    epochs = 100

    for epoch in range(epochs):
        # Train the model for one epoch
        history = model.fit(x_train, y_train, epochs=1, validation_data=(x_test,y_test), verbose=0)

        # Record loss and accuracy values for plotting
        loss_history.append(history.history['loss'][0])
        accuracy_history.append(history.history['accuracy'][0])

        # Display the loss and accuracy for the current epoch
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss_history[-1]:.4f} - Accuracy: {accuracy_history[-1]:.4f}")

        # Plot loss and accuracy
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(loss_history)
        plt.title('Loss vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.subplot(1, 2, 2)
        plt.plot(accuracy_history)
        plt.title('Accuracy vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.tight_layout()
        plt.show()

    """
    history = model.fit(x_train, y_train,  batch_size=32, epochs=50,shuffle=True,validation_data=(x_test,y_test))
    #history = model.fit(x_train, y_train,  batch_size=32)

    loss, acc = model.evaluate(x_test, y_test)
    print("Accuracy: {:.2f}%".format(acc*100))
    print("Loss: {:.2f}%".format(loss*100))
    print("Highest Accuracy: {:.2f}%".format(max(history.history['val_accuracy'])*100))
    """

    try:
      #import pickle
      #pickle.dump(model, open("/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.pkl", "wb"))

      #from joblib import dump, load
      #dump(model, "/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.joblib")

      model.save('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.tf')
      #model.save('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.keras')
      #tf.saved_model.save(model, '/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.tf')
      print("Model Saved")
    except Exception as e:
      print(f"An exception occurred: {str(e)}")

    """
    try:
      #model.save_weights(filepath='/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets_weights.tf', overwrite=True, save_format=tf, options=None)
      model.save_weights(filepath='/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets_weights')
      print("Weights Saved")
    except Exception as e:
      print(f"An exception occurred: {str(e)}")
    """

    print("Dendritic_CapsNets Done")

if __name__ == "__main__":
    #initialise()
    Dendritic_CapsNets_example()
    folder_size()

*************START*************



FileNotFoundError: ignored